In [3]:
%matplotlib inline
import pylab as pl
import numpy as np
import scipy.io.wavfile as wavf
import os
import math
import sys

def readWavFile(name):
    frequencySample, signal = wavf.read(name)
    return frequencySample, signal

def preprocessSignal(signal):
    signal = makeMono(signal)
    signal = removeSilence(signal)
    return signal

def makeMono(signal):
    finalSignal = []
    for s in signal:
        if isNumber(s):
            finalSignal.append(s)
        else:
            stereoMean = np.int16((s[0] + s[1]) / 2)
            finalSignal.append(stereoMean)
            
    return np.array(finalSignal)

def isNumber(value):
    return type(value) is np.int16

def removeSilence(signal):
    finalSignal = []
    threshold = 400
    for s in signal:
        if abs(s) > threshold:
            finalSignal.append(s)
    
    return np.array(finalSignal)

def top_peaks(signal, freqSample, num = 10):
    freqs = pl.fftfreq(len(signal))
    freqs = freqs * freqSample
    for i in range(len(freqs)):
        if freqs[i]<50:
            signal[i]=0
        elif freqs[i]>250:
            signal[i]=0            
    peaks = []            
    for i in range(num):
        k = pl.argmax(signal)
        freq = freqs[k]
        peaks.append(freq)
        signal[k]=0        
    return peaks

def check_voice(wav_name):
    freqSample, signal = readWavFile('train/' + wav_name)
    signal = preprocessSignal(signal)
    peaks = top_peaks(signal, freqSample)
    avr = np.mean(peaks)
    if avr > 160:
        print(wav_name + ': ---K---: ' + str(avr))
    elif avr > 50 and avr < 130:
        print(wav_name + ': ---M---: ' + str(avr))
    else:
        print(wav_name + ': ---?---: ' + str(avr))

if __name__ == '__main__':
#     frequencySample, signal = readWavFile(sys.argv[0]) # final version
#    freqSample, signal = readWavFile('train/005_M.wav') # test version
    for wav_file in os.listdir('train'):
        print(check_voice(wav_file))
    pl.plot(signal)
    # pl.plot(signal[:,0], signal[:,1])
    # pl.plot(signal[:,0])
    # pl.plot(signal[:,1])

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:24: RuntimeWarning: overflow encountered in short_scalars


057_K.wav: ---K---: 180.518057421
None
064_M.wav: ---M---: 83.2385701636
None
076_M.wav: ---M---: 56.8459697596
None


/usr/lib/python3/dist-packages/scipy/io/wavfile.py:42: WavFileWarning: Unknown wave file format
  warnings.warn("Unknown wave file format", WavFileWarning)


082_M.wav: ---M---: 65.7266336096
None
035_M.wav: ---M---: 126.919766663
None
046_K.wav: ---K---: 180.917880373
None
042_M.wav: ---?---: 0.0
None
005_M.wav: ---M---: 113.761698781
None
020_M.wav: ---M---: 109.418969849
None
017_M.wav: ---K---: 165.879920751
None
026_M.wav: ---K---: 161.662404092
None
078_M.wav: ---K---: 191.231048776
None
044_K.wav: ---?---: 0.0
None
030_M.wav: ---?---: 139.77384263
None
037_K.wav: ---?---: 0.0
None
029_K.wav: ---K---: 191.580183559
None


/usr/lib/python3/dist-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


072_K.wav: ---M---: 94.5006490429
None
018_K.wav: ---?---: 147.959033737
None
067_K.wav: ---K---: 224.993787641
None
021_M.wav: ---M---: 101.947292556
None
038_M.wav: ---K---: 221.136145075
None
081_K.wav: ---M---: 64.9248663685
None
055_K.wav: ---?---: 137.669305674
None
068_K.wav: ---?---: 130.922497966
None
049_M.wav: ---M---: 98.5071374087
None
034_K.wav: ---K---: 172.009091445
None
024_M.wav: ---K---: 245.08532931
None
059_K.wav: ---?---: 0.0
None
073_K.wav: ---K---: 167.172926349
None
065_M.wav: ---?---: 0.0
None
071_M.wav: ---K---: 201.517683875
None


ZeroDivisionError: float division by zero